# Museums in the Pandemic - Text analytics

**Author**: Andrea Ballatore (Birkbeck, University of London)

**Abstract**: TODO

## Setup
This is to check that your environment is set up correctly (it should print 'env ok', ignore warnings).

In [1]:
# Test geospatial libraries
# check environment
import os
print("Conda env:", os.environ['CONDA_DEFAULT_ENV'])
if os.environ['CONDA_DEFAULT_ENV'] != 'mip_v1':
    raise Exception("Set the environment 'mip_v1' on Anaconda. Current environment: " + os.environ['CONDA_DEFAULT_ENV'])

# spatial libraries 
import pandas as pd
import pickle
import sys
import numpy as np
#import tensorflow as tf
from bs4 import BeautifulSoup
from bs4.element import Comment
#import torch
import matplotlib.pyplot as plt

# import from `mip` project
print(os.getcwd())
fpath = os.path.abspath('../')
if not fpath in sys.path:
    sys.path.insert(0, fpath)

print('env ok')

Conda env: mip_v1
/Users/andreab/Dropbox/DRBX_Docs/Work/Projects/github_projects/museums-in-the-pandemic/mip/notebooks
env ok


# Museum text analytics


## Vectorise text from museum websites

In [2]:
# open connection to DB
from db.db import connect_to_postgresql_db

db_conn = connect_to_postgresql_db()
print(db_conn)

<connection object at 0x7fc921a3feb0; dsn: 'user=postgres password=xxx connect_timeout=3 dbname=mip host=193.61.36.75', closed: 0>


In [3]:
from analytics.an_websites import get_attribute_for_webpage_url

session_id = '20210420'
test_urls = ['https://www.britishmuseum.org/']
attrib_name = 'all_text' # 'title'

for url in test_urls:
    print(url)
    res = get_attribute_for_webpage_url(url, session_id, attrib_name, db_conn)
    if not res: continue
    blocks = res.split("\n")
    print("LEN", len(blocks))
    print(blocks)
    
    # TODO: segment sentences
    
    # TODO: remove stopwords 
    
    # TODO: lemmatise  
    
    

https://www.britishmuseum.org/
LEN 57
["Skip to main content Please enable JavaScript in your web browser to get the best experience. We use cookies to make our website work more efficiently, to provide you with more personalised services or advertising to you, and to analyse traffic on our website. For more information on how we use cookies and how to manage cookies, please follow the 'Read more' link, otherwise select 'Accept and close'. Read more about our cookie policy Accept and close the cookie policy Menu Main navigation Visit Toggle Visit submenu Back", 'to previous menu —', 'Visit —', 'Visit —', 'Family visits —', 'Group visits —', 'Audio guide —', 'Out-of-hours tours —', 'Tours and talks —', 'Object trails —', 'Accessibility —', 'Food and drink —', 'Late opening on Fridays —', 'Museum map Exhibitions and events Collection Toggle Collection submenu Back', 'to previous menu —', 'Collection —', 'Collection —', 'Collection online —', 'Galleries —', 'Blog —', 'Audio tour highlight

End of notebook